In [1]:
import requests
import xmltodict
import datetime
import pandas as pd
import glob
import re

In [2]:
# 현재 날짜 확인
now = datetime.datetime.now()
nowDate = now.strftime('%Y%m%d')

nowDate

'20210524'

In [3]:
# 현재 디렉토리에 있는 가장 최신 파일 날짜 확인
ls_all = glob.glob("./dataset/all/*.csv")
if not ls_all:
    oldDate_all = '20200101'
else:
    ls_all.sort()
    oldDate_all = ls_all[-1][22:-8]
    oldDate_csv_all = ls_all[-1][14:]

ls_part = glob.glob("./dataset/part/*.csv")
if not ls_part:
    oldDate_part = '20200101'
else:
    ls_part.sort()
    oldDate_part = ls_part[-1][23:-9]
    oldDate_csv_part = ls_part[-1][15:]

oldDate_all, oldDate_csv_all, oldDate_part, oldDate_csv_part

# all: 전국 통합, part: 시, 도 분할

('20210512',
 'covid19_20210512_all.csv',
 '20210512',
 'covid19_20210512_part.csv')

In [4]:
# 현재 날짜부터 디렉토리에 있는 가장 최신 파일 날짜까지 데이터 다운로드
url_base_all = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
url_base_part = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url_serviceKey = 'j8RHAoYLh5NFcdYjbRbdSoOGiuOeHwQZZaU3Mqu78SUYGddoObBnh0Xx3o%2BVPFtZhz5hZaQOf%2B9dMFljJK%2BpLQ%3D%3D'
url_pages = '1000'
url_start_date_all = oldDate_all
url_start_date_part = oldDate_part
url_end_date = nowDate
url_all = url_base_all + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date_all + '&endCreateDt=' + url_end_date
url_part = url_base_part + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date_part + '&endCreateDt=' + url_end_date

In [5]:
req_all = requests.get(url_all).content
req_part = requests.get(url_part).content
xmlObject_all = xmltodict.parse(req_all)
xmlObject_part = xmltodict.parse(req_part)
dict_data_all = xmlObject_all['response']['body']['items']['item']
dict_data_part = xmlObject_part['response']['body']['items']['item']

In [6]:
df_new_all = pd.DataFrame(dict_data_all)
df_new_part = pd.DataFrame(dict_data_part)

In [7]:
for i in range(df_new_part.shape[0]):
    t = re.split(r'년|월|일|시| ', df_new_part['stdDay'][i])
    df_new_part['stdDay'][i] = ''.join(t[0:5:2])

df_new_part

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-05-24 09:46:24.361,4,3765,검역,隔離區,Lazaretto,11,3538,223,0,11,-,9716,20210524,null
1,2021-05-24 09:46:24.361,1,946,제주,济州,Jeju,19,830,115,19,0,141.04,9715,20210524,null
2,2021-05-24 09:46:24.361,19,4580,경남,庆南,Gyeongsangnam-do,17,4302,259,17,0,136.26,9714,20210524,null
3,2021-05-24 09:46:24.361,83,4605,경북,庆北,Gyeongsangbuk-do,12,4232,290,12,0,172.96,9713,20210524,null
4,2021-05-24 09:46:24.36,12,1402,전남,全南,Jeollanam-do,14,1167,223,12,2,75.19,9712,20210524,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-05-12 09:51:43.258,60,5859,인천,仁川,Incheon,20,5616,183,20,0,198.20,9455,20210512,null
243,2021-05-12 09:51:43.258,221,9454,대구,大邱,Daegu,6,9129,104,5,1,388.02,9454,20210512,null
244,2021-05-12 09:51:43.257,122,5296,부산,釜山,Busan,10,4923,251,10,0,155.23,9453,20210512,null
245,2021-05-12 09:51:43.257,466,40118,서울,首尔,Seoul,223,36890,2762,220,3,412.16,9452,20210512,null


In [8]:
df_new_all = df_new_all.sort_values(by=['stateDt'])
df_new_part = df_new_part.sort_values(by=['stdDay'])
df_new_all = df_new_all.drop(df_new_all.index[0:1])
df_new_part = df_new_part.drop(df_new_part.index[0:20])

In [9]:
df_new_all

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
11,1.4161913364,9241153,9153636,7836,119906,2021-05-13 09:41:10.7,1891,129633,87517,9024003,510,20210513,00:00,null
10,1.4193209445,9281291,9186083,8092,120395,2021-05-14 09:44:47.494,1893,130380,95208,9055703,511,20210514,00:00,null
9,1.4215047097,9319094,9219878,7883,121282,2021-05-15 09:38:53.579,1896,131061,99216,9088817,512,20210515,00:00,null
8,1.4257899148,9336614,9234951,8007,121764,2021-05-16 09:26:31.589,1900,131671,101663,9103280,513,20210516,00:00,null
7,1.4299308998,9353300,9251496,8224,122163,2021-05-17 09:39:34.169,1903,132290,101804,9119206,514,20210517,00:00,null
6,1.4294430308,9395004,9291591,8283,122631,2021-05-18 09:41:35.393,1904,132818,103413,9158773,515,20210518,00:00,null
5,1.4314548779,9428644,9324150,8322,123237,2021-05-19 09:39:48.186,1912,133471,104494,9190679,516,20210519,00:00,null
4,1.4361751397,9447358,9338485,8542,123659,2021-05-20 09:35:45.542,1916,134117,108873,9204368,517,20210520,00:00,2021-05-21 09:47:58.875
3,1.4368773847,9487108,9372964,8598,124158,2021-05-21 09:36:32.023,1922,134678,114144,9238286,518,20210521,00:00,null
2,1.4393079615,9520966,9403408,8386,125032,2021-05-22 09:37:01.885,1926,135344,117558,9268064,519,20210522,00:00,null


In [10]:
df_new_part
#   속성 설명(all)
#   DECIDE_CNT          확진자수
#   CLEAR_CNT           격리해제 수
#   EXAM_CNT            검사진행 수
#   DEATH_CNT           사망자 수
#   CARE_CNT            치료중 환자 수
#   RESULT_NEG_CNT      결과 음성 수
#   ACC_EXAM_CNT        누적 검사 수
#   ACC_EXAM_COMP_CNT   누적 검사 완료 수
#   ACC_DEF_RATE        누적 확진률
#   CREATE_DT           등록일시분초
#   UPDATE_DT           수정일시분

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
210,2021-05-13 09:50:30.363,1,838,제주,济州,Jeju,9,714,123,9,0,124.93,9487,20210513,null
211,2021-05-13 09:50:30.363,19,4316,경남,庆南,Gyeongsangnam-do,32,3999,298,30,2,128.40,9486,20210513,null
212,2021-05-13 09:50:30.363,79,4382,경북,庆北,Gyeongsangbuk-do,23,4029,274,22,1,164.58,9485,20210513,null
213,2021-05-13 09:50:30.363,12,1207,전남,全南,Jeollanam-do,12,1039,156,12,0,64.73,9484,20210513,null
214,2021-05-13 09:50:30.363,58,2065,전북,全北,Jeollabuk-do,10,1892,115,10,0,113.63,9483,20210513,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2021-05-24 09:46:24.358,124,5555,부산,釜山,Busan,15,5169,262,14,1,162.82,9700,20210524,null
17,2021-05-24 09:46:24.358,480,42601,서울,首尔,Seoul,139,39276,2845,136,3,437.67,9699,20210524,null
18,2021-05-24 09:46:24.358,1934,136467,합계,合计,Total,538,126427,8106,513,25,263.21,9698,20210524,null
9,2021-05-24 09:46:24.36,628,38008,경기,京畿,Gyeonggi-do,122,35278,2102,120,2,286.84,9707,20210524,null


In [11]:
df_old_all = pd.read_csv('./dataset/all/covid19_' + oldDate_all + '_all.csv')
df_old_all = df_old_all.drop(['Unnamed: 0'], axis = 1)
df_old_part = pd.read_csv('./dataset/part/covid19_' + oldDate_part + '_part.csv')
df_old_part = df_old_part.drop(['Unnamed: 0'], axis = 1)
df_old_all

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,NaN,NaN,NaN,NaN,1,47:33.3,0,0,1,NaN,1,20200101,18:00,21:56.6
1,NaN,NaN,NaN,NaN,1,47:33.3,0,0,1,NaN,1,20200201,18:00,47:33.3
2,NaN,NaN,NaN,NaN,2,22:49.5,2,2,2,NaN,2,20200202,9:00,22:49.5
3,NaN,NaN,NaN,NaN,0,41:17.2,0,15,0,NaN,3,20200203,9:00,41:17.2
4,NaN,NaN,NaN,NaN,0,26:59.6,0,0,0,NaN,4,20200204,0:00,26:59.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,1.406573,9089166.0,9010907.0,7999.0,116881,2021-05-08 09:37:54.141,1865,126745,78259,8884162.0,505,20210508,00:00,NaN
464,1.410706,9105771.0,9024491.0,8012.0,117423,2021-05-09 09:38:01.137,1874,127309,81280,8897182.0,506,20210509,00:00,NaN
465,1.413616,9122059.0,9038661.0,8053.0,117844,2021-05-10 09:36:30.35,1875,127772,83398,8910889.0,507,20210510,00:00,NaN
466,1.412862,9164284.0,9079656.0,7687.0,118717,2021-05-11 09:39:41.186,1879,128283,84628,8951373.0,508,20210511,00:00,NaN


In [12]:
df_all = df_old_all.append(df_new_all)
df_part = df_old_part.append(df_new_part)
df_all.to_csv('./dataset/all/covid19_' + nowDate + '_all.csv', mode='w')
df_part.to_csv('./dataset/part/covid19_' + nowDate + '_part.csv', mode='w')

In [13]:
df_all.info()















<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 0
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   accDefRate      449 non-null    object
 1   accExamCnt      449 non-null    object
 2   accExamCompCnt  449 non-null    object
 3   careCnt         448 non-null    object
 4   clearCnt        480 non-null    object
 5   createDt        480 non-null    object
 6   deathCnt        480 non-null    object
 7   decideCnt       480 non-null    object
 8   examCnt         480 non-null    object
 9   resutlNegCnt    449 non-null    object
 10  seq             480 non-null    object
 11  stateDt         480 non-null    object
 12  stateTime       480 non-null    object
 13  updateDt        431 non-null    object
dtypes: object(14)
memory usage: 56.2+ KB
